In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error
from xgboost import XGBRegressor

import pickle
import joblib

In [9]:
df = pd.read_csv('VN_housing_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
print(df.shape)
print(df.info())
df.head()

(82497, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82497 entries, 0 to 82496
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Ngày             82496 non-null  object
 1   Địa chỉ          82449 non-null  object
 2   Quận             82495 non-null  object
 3   Huyện            82449 non-null  object
 4   Loại hình nhà ở  82465 non-null  object
 5   Giấy tờ pháp lý  53610 non-null  object
 6   Số tầng          36399 non-null  object
 7   Số phòng ngủ     82458 non-null  object
 8   Diện tích        82495 non-null  object
 9   Dài              19827 non-null  object
 10  Rộng             35445 non-null  object
 11  Giá/m2           82484 non-null  object
dtypes: object(12)
memory usage: 7.6+ MB
None


,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [10]:
df.columns = ['ngay', 'diachi', 'quan', 'huyen', 'loaihinhnhao', 'giaytophaply', 'sotang', 'sophongngu', 'dientich', 'dai', 'rong', 'dongia']
df = df[df.dongia.notna()]
print(df.shape[0])
df

82484


,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"
...,...,...,...,...,...,...,...,...,...,...,...,...
82491,2019-08-23,"Đường Hồ Tùng Mậu, Phường Phúc Diễn, Quận Bắc ...",Quận Bắc Từ Liêm,Phường Phúc Diễn,Nhà phố liền kề,NaN,NaN,3 phòng,38 m²,NaN,NaN,"81,58 triệu/m²"
82492,2019-08-07,"Đường Trần Quốc Hoàn, Phường Quan Hoa, Quận Cầ...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,50 m²,NaN,NaN,292 triệu/m²
82493,2019-08-07,"Đường Nguyễn Khánh Toàn, Phường Quan Hoa, Quận...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà mặt phố, mặt tiền",Đã có sổ,NaN,4 phòng,41 m²,NaN,NaN,"341,46 triệu/m²"
82494,2019-08-05,"Đường Quan Hoa, Phường Quan Hoa, Quận Cầu Giấy...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà ngõ, hẻm",Đã có sổ,NaN,4 phòng,60 m²,NaN,NaN,"101,67 triệu/m²"


In [11]:
df = df[df['sotang'] != 'Nhiều hơn 10']
df['sotang'] = df['sotang'].fillna(0)
df = df[df['sophongngu'] != 'nhiều hơn 10 phòng']
print(df.shape[0])

81615


In [12]:
df['duong'] = df['diachi'].str.split(', ', expand=True)[0]
df['sotang'] = df['sotang'].astype(int)
df['sophongngu'] = df['sophongngu'].str.replace(' phòng','').str.strip().astype(float)
df['dientich'] = df['dientich'].str.split('m', expand=True)[0].astype(float)
df['dai'] = df['dai'].str.split('m', expand=True)[0].astype(float)
df['rong'] = df['rong'].str.split('m', expand=True)[0].astype(float)
print(df.shape[0] == 81615)
df.head()

True


,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia,duong
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,"86,96 triệu/m²",Đường Hoàng Quốc Việt
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,0,3.0,37.0,NaN,NaN,"116,22 triệu/m²",Đường Kim Giang
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,40.0,10.00,4.0,65 triệu/m²,phố minh khai
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,0,6.0,51.0,12.75,4.0,100 triệu/m²,Đường Võng Thị
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,0,4.0,36.0,9.00,4.0,"86,11 triệu/m²",Đường Kim Giang


In [13]:
# df['dongia'] = df['dongia'].replace([','], ['.'])
# df

In [14]:
# Clean and convert all prices to million/m2 instead of VND/m2 or billion/m2
df.loc[df['dongia'].str.contains(' tỷ/m²'), 'dongia'] = df.loc[df['dongia'].str.contains(' tỷ/m²'), 'dongia'].str.replace(' tỷ/m²','').str.replace('.','').str.replace(',','.').astype(float) * 1000
df.loc[df['dongia'].str.contains(' triệu/m²', na=False), 'dongia'] = df.loc[df['dongia'].str.contains(' triệu/m²', na=False), 'dongia'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df.loc[df['dongia'].str.contains(' đ/m²', na=False), 'dongia'] = df.loc[df['dongia'].str.contains(' đ/m²', na=False), 'dongia'].str.replace(' đ/m²','').str.replace('.','').astype(float) * 0.000001
# 4. Cuối cùng: chuyển toàn bộ cột sang float
df['dongia'] = pd.to_numeric(df['dongia'], errors='coerce')
df.head()

,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia,duong
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96,Đường Hoàng Quốc Việt
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,0,3.0,37.0,NaN,NaN,116.22,Đường Kim Giang
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,40.0,10.00,4.0,65.00,phố minh khai
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,0,6.0,51.0,12.75,4.0,100.00,Đường Võng Thị
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,0,4.0,36.0,9.00,4.0,86.11,Đường Kim Giang


In [15]:
num_cols = []
cat_cols = []
for col in df.columns:
    if col != 'dongia':
        if df[col].dtype == 'O':
            cat_cols.append(col)
        else:
            num_cols.append(col)
        
print(num_cols)
print()
print(cat_cols)

['sotang', 'sophongngu', 'dientich', 'dai', 'rong']

['ngay', 'diachi', 'quan', 'huyen', 'loaihinhnhao', 'giaytophaply', 'duong']


In [16]:
label = 'dongia'
print(num_cols)
print()
cat_cols = cat_cols[2:]
print(cat_cols)
features = cat_cols + num_cols
features

['sotang', 'sophongngu', 'dientich', 'dai', 'rong']

['quan', 'huyen', 'loaihinhnhao', 'giaytophaply', 'duong']


['quan',
 'huyen',
 'loaihinhnhao',
 'giaytophaply',
 'duong',
 'sotang',
 'sophongngu',
 'dientich',
 'dai',
 'rong']

In [17]:
print(df.ngay.min())
print(df.ngay.max())

2019-08-05
2020-08-05


In [18]:
# inter quantile range |---------Q1---------Q2-----------Q3------------|      ---
#                                 -------------------------   => IQR, any point < Q1 - 1.5*IQR, any point > Q3 + 1.5IQR  

## feature1, feature2, feature3, outlier cho tung feature, , neu row_index ma xay ra bat thuong it nhat n cot ==> nhieu kha nang
from collections import Counter

def detect_outliers(df,n,features):
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col],25)
        
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers fro feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index

        # append the found outlier indices for col to the list of outlier indices
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than n outliers
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k,v in outlier_indices.items() if v > n)
    
    return multiple_outliers

outlier_to_drop = detect_outliers(df, 2, num_cols)
outlier_to_drop

[]

In [19]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [20]:
train.loc[outlier_to_drop]

,ngay,diachi,quan,huyen,loaihinhnhao,giaytophaply,sotang,sophongngu,dientich,dai,rong,dongia,duong


In [21]:
train = train.drop(outlier_to_drop, axis = 0).reset_index(drop=True)

In [22]:
X_train, y_train, X_test, y_test = train[features], train[label], test[features], test[label]
print(train.shape[0] == X_train.shape[0])
print(test.shape[0] == X_test.shape[0])
X_train

True
True


,quan,huyen,loaihinhnhao,giaytophaply,duong,sotang,sophongngu,dientich,dai,rong
0,Quận Bắc Từ Liêm,Phường Cổ Nhuế 1,"Nhà ngõ, hẻm",Đã có sổ,Đường Trần Cung,0,5.0,52.0,11.0,5.0
1,Quận Thanh Xuân,Phường Khương Trung,"Nhà ngõ, hẻm",NaN,Đường Khương Trung,0,10.0,46.0,NaN,NaN
2,Quận Cầu Giấy,Phường Quan Hoa,"Nhà mặt phố, mặt tiền",NaN,Đường Hoàng Quốc Việt,0,6.0,50.0,NaN,4.0
3,Quận Long Biên,Phường Việt Hưng,Nhà biệt thự,Đã có sổ,việt hưng,3,7.0,320.0,NaN,20.0
4,Quận Cầu Giấy,Phường Nghĩa Tân,"Nhà mặt phố, mặt tiền",NaN,Đường Phùng Chí Kiên,0,7.0,80.0,11.0,7.0
...,...,...,...,...,...,...,...,...,...,...
65287,Quận Hà Đông,Phường Kiến Hưng,Nhà phố liền kề,Đã có sổ,Phúc la,5,5.0,83.0,12.0,7.0
65288,Quận Thanh Xuân,Phường Khương Mai,"Nhà ngõ, hẻm",Đã có sổ,Đường Lê Trọng Tấn,5,3.0,32.0,NaN,NaN
65289,Quận Tây Hồ,Phường Xuân La,"Nhà ngõ, hẻm",NaN,Đường Xuân La,0,4.0,52.0,NaN,NaN
65290,Quận Hoàng Mai,Phường Định Công,"Nhà ngõ, hẻm",Đã có sổ,Đường Định Công||956,5,4.0,44.0,NaN,NaN


In [23]:
def Log_Dientich_Transform(dientich):
    return np.log1p(dientich)
Log_Dientich_Transformer = FunctionTransformer(Log_Dientich_Transform, validate=False)

class TotalSoPhongTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Assume X is a DataFrame with 'price' and 'sqft' columns
        X = X.copy()
        X['total_sophong'] = X['sophongngu'] * X['sotang']
        return X[['total_sophong']]

In [ ]:
# # Create column transformer for OHE
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("log_dientich", Log_Dientich_Transformer, ['dientich']),
#         ("total_sophong", TotalSoPhongTransformer(), ['sophongngu', 'sotang']),
#         ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols),
#         ("scale", StandardScaler(), num_cols)
#     ]
# )

# # Create a pipeline with preprocessing and random forest classifier
# pipeline_xgb = Pipeline(steps=[
#     ("preprocessor", preprocessor),
#     ("regressor", XGBRegressor(objective='reg:squarederror', random_state=42))
# ])

# # Define hyperparameter grid
# param_grid = {
#     "regressor__n_estimators": [100, 200],
#     "regressor__max_depth": [3, 5, 7],
#     "regressor__learning_rate": [0.01, 0.1, 0.2],
#     "regressor__subsample": [0.8, 1.0],
#     "regressor__colsample_bytree": [0.8, 1.0]
# }

# # Grid search setup
# grid_search_pipeline = GridSearchCV(
#     estimator=pipeline_xgb,
#     param_grid=param_grid,
#     cv=5,
#     scoring='neg_mean_absolute_percentage_error',  # or 'r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'
#     verbose=2,
#     n_jobs=-1
# )

# # Fit to training data
# grid_search_pipeline.fit(X_train, y_train)

In [ ]:
# # Results
# print("✅ Best parameters:", grid_search_pipeline.best_params_)
# print("📉 Best RMSE:", grid_search_pipeline.best_score_)

In [26]:
# # Giả sử grid_search là đối tượng GridSearchCV đã fit
# joblib.dump(grid_search_pipeline, "grid_search_pipeline_model.pkl")

In [27]:
# del grid_search_pipeline

In [ ]:
# Với joblib
grid_search_pipeline = joblib.load("grid_search_pipeline_model.pkl")

In [41]:
y_train_pred = grid_search_pipeline.predict(X_train)
print(r2_score(y_train_pred, y_train))
print(mean_absolute_percentage_error(y_train_pred, y_train))

0.9999999981443491
0.2469513863989205


In [42]:
y_test_pred = grid_search_pipeline.predict(X_test)
print(r2_score(y_test_pred, y_test))
print(mean_absolute_percentage_error(y_test_pred, y_test))

-24.288062518674778
55.570276170928096


In [35]:
# # Giả sử grid_search là đối tượng GridSearchCV đã fit
# joblib.dump(grid_search_pipeline, "grid_search_pipeline_model.pkl")